In [1]:
from sqlalchemy import *
from sqlalchemy.sql import *

import pandas
from matplotlib import pyplot as plt

engine = create_engine('sqlite:///Data/Chinook_Sqlite.sqlite')
metadata = MetaData()
metadata.reflect(bind=engine)
conn=engine.connect()

Album=Table("Album", metadata, autoload=True, autoload_with="engine")
MediaType=Table("MediaType", metadata, autoload=True, autoload_with="engine")
Artist=Table("Artist", metadata, autoload=True, autoload_with="engine")
Customer=Table("Customer", metadata, autoload=True, autoload_with="engine")
Employee=Table("Employee", metadata, autoload=True, autoload_with="engine")
Track=Table("Track", metadata, autoload=True, autoload_with="engine")
Genre=Table("Genre", metadata, autoload=True, autoload_with="engine")
Invoice=Table("Invoice", metadata, autoload=True, autoload_with="engine")
InvoiceLine=Table("InvoiceLine", metadata, autoload=True, autoload_with="engine")
Playlist=Table("Playlist", metadata, autoload=True, autoload_with="engine")
PlaylistTrack=Table("PlaylistTrack", metadata, autoload=True, autoload_with="engine")
A = Album.alias('A')
M = MediaType.alias('M')
Ar = Artist.alias('Ar')
C = Customer.alias('C')
E = Employee.alias('E')
T = Track.alias('T')
G = Genre.alias('G')
I = Invoice.alias('I')
IL = InvoiceLine.alias('IL')
P = Playlist.alias('P')
PT = PlaylistTrack.alias('PT')

print(metadata.tables.keys())

import pandas as pd
df=pd.read_csv('./Data/imdb.csv')

dict_keys(['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'Track', 'MediaType', 'Playlist', 'PlaylistTrack'])


#### №1. Соединение таблиц с помощью оператора JOIN. Операции соединения таблиц посредством ссылочной целостности.
Оператор языка SQL JOIN предназначен для соединения двух или более таблиц базы данных по совпадающему условию. Этот оператор существует только в реляционных базах данных. Именно благодаря JOIN реляционные базы данных обладают такой мощной функциональностью, которая позволяет вести не только хранение данных, но и их, хотя бы простейший, анализ с помощью запросов.

Для соединения двух таблиц оператор SQL JOIN имеет следующий синтаксис:

SELECT ИМЕНА_СТОЛБЦОВ 
FROM ИМЯ_ТАБЛИЦЫ_1 JOIN ИМЯ_ТАБЛИЦЫ_2 
ON УСЛОВИЕ

Запрос с оператором LEFT OUTER JOIN предназначен для соединения таблиц и вывода результирующей таблицы, в которой данные полностью пересекаются по условию, указанному после ON, и дополняются записями из первой по порядку (левой) таблицы, даже если они не соответствуют условию. У записей левой таблицы, которые не соответствуют условию, значение столбца из правой таблицы будет NULL (неопределённым).

Запрос с оператором RIGHT OUTER JOIN предназначен для соединения таблиц и вывода результирующей таблицы, в которой данные полностью пересекаются по условию, указанному после ON, и дополняются записями из второй по порядку (правой) таблицы, даже если они не соответствуют условию. У записей правой таблицы, которые не соответствуют условию, значение столбца из левой таблицы будет NULL (неопределённым).

Запрос с оператором FULL OUTER JOIN предназначен для соединения таблиц и вывода результирующей таблицы, в которой данные полностью пересекаются по условию, указанному после ON, и дополняются записями из первой (левой) и второй (правой) таблиц, даже если они не соответствуют условию. У записей, которые не соответствуют условию, значение столбцов из другой таблицы будет NULL (неопределённым).

#### №2. С помощью библиотеки SQLAlchemy создайте объекты таблиц базы данных Chinook. Создайте запрос, который показывает название произведения (Track), имя исполнителя (Artist), идентификатор строки заказа (InvoiceLine) для всех купленных произведений в стиле Classical (Genre). Отсортируйте результат по названию трека.

In [2]:
s = select(T.c.Name, A.c.Title, IL.c.InvoiceLineId, G.c.Name).where(and_(
    T.c.GenreId == G.c.GenreId, 
    I.c.InvoiceId == IL.c.InvoiceId,
    IL.c.TrackId == T.c.TrackId,
    T.c.AlbumId == A.c.AlbumId, 
    G.c.Name == 'Classical')).order_by(T.c.Name)
result = conn.execute(s).fetchall()
ans = pd.DataFrame(result, columns = ['Track Name', 'Album Title', 'Invoice Line Id', 'Genre'])
ans

,Track Name,Album Title,Invoice Line Id,Genre
0,"A Midsummer Night's Dream, Op.61 Incidental Mu...",Mendelssohn: A Midsummer Night's Dream,1149,Classical
1,"Act IV, Symphony",Purcell: The Fairy Queen,573,Classical
2,Adios nonino,South American Getaway,572,Classical
3,Canon and Gigue in D Major: I. Canon,Pachelbel: Canon & Gigue,1133,Classical
4,"Concerto No.2 in F Major, BWV1047, I. Allegro",Bach: The Brandenburg Concertos,1709,Classical
5,"Concerto for Cello and Orchestra in E minor, O...",Elgar: Cello Concerto & Vaughan Williams: Fant...,1713,Classical
6,"Concerto for Piano No. 2 in F Minor, Op. 21: I...",Chopin: Piano Concertos Nos. 1 & 2,1710,Classical
7,Die Walküre: The Ride of the Valkyries,Wagner: Favourite Overtures,1707,Classical
8,Fantasia On Greensleeves,The World of Classical Favourites,1712,Classical
9,"Jupiter, the Bringer of Jollity","Holst: The Planets, Op. 32 & Vaughan Williams:...",1135,Classical


#### №3. Записи набора данных imdb.csv содержат информацию о фильмах. Атрибут Genre каждой записи содержит перечень жанров конкретного фильма. Найдите среднее значение балльной оценки (атрибут Metascore) всех фильмов в жанре Action.

In [3]:
print(f'Cреднее значение балльной оценки: \
{df[df["Genre"].str.contains("Action")]["Metascore"].mean()}')

Cреднее значение балльной оценки: 53.83044982698962
